In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

2024-08-04 18:56:29.010972: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 18:56:29.648681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-04 18:56:29.648776: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-08-04 18:56:29.648782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

tf.keras.utils.set_random_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [4]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(439, 72)
(439, 17)
(94, 72)
(94, 17)
(95, 72)
(95, 17)


In [5]:
#build model

def define_model(seedval):
    inputs = tf.keras.Input(shape=(72, 1), name='input')
    x = Conv1D(filters=5, kernel_size=3,strides =3)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Conv1D(filters=45, kernel_size=5,strides =2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dropout(0.5,seed=seedval)(x)
    x = Dense(20,activation ='relu')(x)
#     x = Dropout(0.5,seed=seedval)(x)
#     x = Dense(10,activation ='relu')(x)
#     x = Dense(5,activation ='relu')(x)
    outputs = Dense(17, activation='softmax', name='predictions')(x)
    cnn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics=['accuracy'])
    return cnn_model

In [6]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50)
checkpoint_path = 'allamino_model3forpaper.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [7]:
model = define_model(seed_value)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 5)             20        
                                                                 
 batch_normalization (BatchN  (None, 24, 5)            20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 45)             1170  

2024-08-04 18:56:30.385218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-04 18:56:30.385262: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VLSI-CAD): /proc/driver/nvidia/version does not exist
2024-08-04 18:56:30.386368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:

history = model.fit(X_train, y_train,
              batch_size=10,
              verbose=1,
              epochs=500,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                    ,class_weight = {0:4.77,1:4.43,2:36.58,3:48.78,4:24.39,5:29.27,6:29.27,7:16.88,8:16.88,9:15.14,10:36.58,11:16.88,12:29.27,13:36.58,14:36.58,15:48.78,16:36.58}
                   )

Epoch 1/500
40/44 [==========================>...] - ETA: 0s - loss: 54.6720 - accuracy: 0.0625 
Epoch 1: val_accuracy improved from -inf to 0.03158, saving model to allamino_model3forpaper.h5
44/44 [==============================] - 1s 6ms/step - loss: 55.0194 - accuracy: 0.0661 - val_loss: 2.8315 - val_accuracy: 0.0316
Epoch 2/500
42/44 [===========================>..] - ETA: 0s - loss: 50.2661 - accuracy: 0.0643
Epoch 2: val_accuracy improved from 0.03158 to 0.10526, saving model to allamino_model3forpaper.h5
44/44 [==============================] - 0s 2ms/step - loss: 50.1122 - accuracy: 0.0615 - val_loss: 2.8269 - val_accuracy: 0.1053
Epoch 3/500
 1/44 [..............................] - ETA: 0s - loss: 57.1940 - accuracy: 0.0000e+00
Epoch 3: val_accuracy improved from 0.10526 to 0.12632, saving model to allamino_model3forpaper.h5
44/44 [==============================] - 0s 2ms/step - loss: 49.0977 - accuracy: 0.0615 - val_loss: 2.8309 - val_accuracy: 0.1263
Epoch 4/500
 1/44 [....

44/44 [==============================] - 0s 2ms/step - loss: 42.6674 - accuracy: 0.1572 - val_loss: 2.8432 - val_accuracy: 0.0421
Epoch 30/500
 1/44 [..............................] - ETA: 0s - loss: 54.3803 - accuracy: 0.2000
Epoch 30: val_accuracy did not improve from 0.18947
44/44 [==============================] - 0s 2ms/step - loss: 41.7010 - accuracy: 0.1800 - val_loss: 2.7933 - val_accuracy: 0.0737
Epoch 31/500
 1/44 [..............................] - ETA: 0s - loss: 32.8547 - accuracy: 0.2000
Epoch 31: val_accuracy did not improve from 0.18947
44/44 [==============================] - 0s 2ms/step - loss: 41.6242 - accuracy: 0.1800 - val_loss: 2.7853 - val_accuracy: 0.0737
Epoch 32/500
 1/44 [..............................] - ETA: 0s - loss: 43.5638 - accuracy: 0.1000
Epoch 32: val_accuracy did not improve from 0.18947
44/44 [==============================] - 0s 2ms/step - loss: 42.5653 - accuracy: 0.1458 - val_loss: 2.8221 - val_accuracy: 0.0842
Epoch 33/500
 1/44 [.............

In [9]:
model1 = tf.keras.models.load_model('allamino_model3forpaper.h5')
model1.evaluate(X_test,y_test)

3/3 [==============================] - 0s 2ms/step - loss: 2.8266 - accuracy: 0.1383


[2.8265998363494873, 0.13829787075519562]

In [10]:
model1 = tf.keras.models.load_model('allamino_model3forpaper.h5')
model1.evaluate(X_val,y_val)

3/3 [==============================] - 0s 2ms/step - loss: 2.8304 - accuracy: 0.1895


[2.8303720951080322, 0.1894736886024475]

In [11]:
model1 = tf.keras.models.load_model('allamino_model3forpaper.h5')
model1.evaluate(X_train,y_train)

14/14 [==============================] - 0s 910us/step - loss: 2.8159 - accuracy: 0.1868


[2.815934181213379, 0.1867881566286087]